In [1]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import Sequential
from keras.layers import SpatialDropout1D, Embedding, LSTM, Dense
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split


nltk.download('stopwords')
stopwords = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lukakrickovic/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### Ucitavanje dataseta

In [2]:
df = pd.read_csv('../datasets/fake-news/balansed_fake_news_dataset.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49550 entries, 0 to 49549
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    49550 non-null  object
 1   text     49548 non-null  object
 2   subject  49550 non-null  object
 3   date     34190 non-null  object
 4   fake     49550 non-null  bool  
dtypes: bool(1), object(4)
memory usage: 1.6+ MB


#### Ciscenje teksta

In [3]:
df = df.reset_index(drop=True)

def clean_text(text: str) -> str:
    REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
    BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
    STOPWORDS = set(stopwords)
    text = text.lower()
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    text = BAD_SYMBOLS_RE.sub('', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)
    return text

df['text'].dropna()
df = df.loc[df.text.apply(type) != float]
df['text'] = df['text'].apply(clean_text)
df['text'] = df['text'].str.replace('\d+', '')

/var/folders/ny/km1m35px577gl9cs19xd2scm0000gn/T/ipykernel_22187/4199856113.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace('\d+', '')


## LSTM

#### Tokenizacija tekstova i priprema hiperparametara

In [4]:
# Hiperparametri odredjeni na osnovu clanka:
# https://towardsdatascience.com/multi-class-text-classification-with-lstm-1590bee1bd17
MAX_NB_WORDS = 50000
MAX_SEQUENCE_LENGTH = 250
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['text'].values)
word_index = tokenizer.word_index

Posto ni jedan clanak nije apsolutno iste duzine, pogotovo kada se tokenizuje, potrebno je uvesti padding tako da svaki token bude maksimalno MAX_NB_WORDS duzine

In [5]:
X = tokenizer.texts_to_sequences(df['text'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)

Konvertujemo True/False u dummies

In [6]:
Y = pd.get_dummies(df['fake']).values

Train test split parametri takodje odredjeni na osnovu navedenog clanka

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 42)

#### LSTM klasifikator

In [8]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

2022-04-25 13:14:54.326818: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/5
558/558 [==============================] - 161s 285ms/step - loss: 0.2106 - accuracy: 0.9164 - val_loss: 0.1132 - val_accuracy: 0.9632
Epoch 2/5
558/558 [==============================] - 152s 273ms/step - loss: 0.0684 - accuracy: 0.9776 - val_loss: 0.1078 - val_accuracy: 0.9680
Epoch 3/5
558/558 [==============================] - 153s 274ms/step - loss: 0.0302 - accuracy: 0.9902 - val_loss: 0.1230 - val_accuracy: 0.9647
Epoch 4/5
558/558 [==============================] - 153s 275ms/step - loss: 0.0129 - accuracy: 0.9966 - val_loss: 0.1542 - val_accuracy: 0.9649
Epoch 5/5
558/558 [==============================] - 827s 1s/step - loss: 0.0286 - accuracy: 0.9911 - val_loss: 0.1681 - val_accuracy: 0.9571


#### Rezultati

In [9]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

310/310 [==============================] - 8s 25ms/step - loss: 0.1633 - accuracy: 0.9542
Test set
  Loss: 0.163
  Accuracy: 0.954


#### Serijalizacija modela - posto se veoma dugo trenira

In [10]:
model.save('../models/fake_news')

2022-04-25 13:39:17.131395: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ../models/fake_news/assets
